In [1]:
import pandas as pd

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install transformers==4.26.0

In [ ]:
import transformers
print(transformers.__version__)

4.26.0


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
df = pd.read_csv('def_api.csv')
df

class                                            meaning
0          GetCurrentObject  GetCurrentObject function (wingdi.h) The GetCu...
1                ExitThread  Application.ExitThread Method Definition Names...
2                    IsMenu  IsMenu function (winuser.h) Determines whether...
3    WSALookupServiceBeginA  WSALookupServiceBeginA function (winsock2.h) T...
4        DebugActiveProcess  DebugActiveProcess function (debugapi.h) Enabl...
..                      ...                                                ...
671                    hoax  MAGAZINE ARTICLES PsExec The Memory Optimizati...
672                     USD  StatusingDataSet.ProjectsRow.PROJ_OPT_CURRENCY...
673                  adware  How to block/remove adware an... You could blo...
674                hacktool  How Microsoft identifies malware and potential...
675                 rootkit  Rootkits Malware authors use rootkits to hide ...

[676 rows x 2 columns]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class WordMeaningDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=64):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = self.data[idx]["class"]
        meaning = self.data[idx]["meaning"]

        # Tokenize word and meaning together
        inputs = self.tokenizer(word, meaning,
                                padding="max_length",
                                truncation=True,
                                max_length=self.max_length,
                                return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
        }

In [ ]:
# Example dataset
df = pd.read_csv('def_api.csv')
data = []
missed_data_points = []
for i,cla in enumerate(df['class']):
  if isinstance(df['meaning'][i],str):
    data.append({'class':cla,'meaning':df['meaning'][i]})
  else:
    data.append({'class':cla,'meaning':''})
print(data)

[{'class': 'GetCurrentObject', 'meaning': "GetCurrentObject function (wingdi.h) The GetCurrentObject function retrieves a handle to an object of the specified type that has been selected into the specified device context (DC). Syntax Parameters [in] hdc A handle to the DC. [in] type The object type to be queried. Remarks An application can use the GetCurrentObject and GetObject functions.GetColorAdjustment The GetColorAdjustment function retrieves the color adjustment values for the specified device context (DC). GetCurrentObject The GetCurrentObject function retrieves a handle to an object of the specified type that has been selected into the specified device context (DC). GetCurrentPositionEx The GetCurrentPositionEx function.GetColorAdjustment The Ge... GetColorSpace The GetColorSpace function retrieves the handle to the input color space from a specified device context. GetCurrentObject The GetCurrentObject function retrieves a handle to an object of the specified type that has bee

In [ ]:
#tokenize dataset
dataset = WordMeaningDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
import torch.optim as optim
# from transformers.optimization import AdamW
from transformers import BertModel, BertTokenizer
from transformers import AdamW
import torch

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define loss function
loss_fn = torch.nn.MSELoss()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Define dummy target labels (modify this for your task)
        target = torch.zeros_like(logits)

        loss = loss_fn(logits, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, ove

Epoch 1, Loss: 0.016828082225230686


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 2, Loss: 0.002514135090442484


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 3, Loss: 0.0016885299334877773


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 4, Loss: 0.0011966898414196776


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 5, Loss: 0.0011598182366202201


In [ ]:
model.save_pretrained("bert_word_meaning_model")
tokenizer.save_pretrained("bert_word_meaning_model")

('bert_word_meaning_model/tokenizer_config.json',
 'bert_word_meaning_model/special_tokens_map.json',
 'bert_word_meaning_model/vocab.txt',
 'bert_word_meaning_model/added_tokens.json')

In [ ]:
!zip -r bert_word_meaning_model.zip bert_word_meaning_model/


  adding: bert_word_meaning_model/ (stored 0%)
  adding: bert_word_meaning_model/config.json (deflated 48%)
  adding: bert_word_meaning_model/pytorch_model.bin (deflated 7%)
  adding: bert_word_meaning_model/vocab.txt (deflated 53%)
  adding: bert_word_meaning_model/special_tokens_map.json (deflated 42%)
  adding: bert_word_meaning_model/tokenizer_config.json (deflated 45%)


In [ ]:
from google.colab import files
files.download('bert_word_meaning_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import zipfile
import os

zip_filename = "bert_word_meaning_model.zip"

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("bert_word_meaning_model")  # You can change the folder name if needed

# List extracted files
os.listdir("bert_word_meaning_model")



['bert_word_meaning_model']

In [11]:
from transformers import BertForSequenceClassification
import torch.optim as optim
# from transformers.optimization import AdamW
from transformers import BertModel, BertTokenizer
from transformers import AdamW
import torch


model = BertForSequenceClassification.from_pretrained("bert_word_meaning_model/bert_word_meaning_model")
tokenizer = BertTokenizer.from_pretrained("bert_word_meaning_model/bert_word_meaning_model")

In [12]:
tokenizer

BertTokenizer(name_or_path='bert_word_meaning_model/bert_word_meaning_model', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [13]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class SentenceEncoder(nn.Module):
    def __init__(self, bert_model_name="bert_word_meaning_model/bert_word_meaning_model", embed_dim=768, num_filters=128, kernel_size=3):
        super(SentenceEncoder, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)  # BERT as embedding layer
        self.conv1d = nn.Conv1d(in_channels=embed_dim, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.relu = nn.ReLU()
        self.max_pool = nn.AdaptiveMaxPool1d(1)  # Global max pooling

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT during encoding
            bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        embeddings = bert_outputs.last_hidden_state  # (batch_size, seq_length, embed_dim)
        embeddings = embeddings.permute(0, 2, 1)  # Convert to (batch_size, embed_dim, seq_length) for CNN

        conv_output = self.conv1d(embeddings)  # Apply CNN
        conv_output = self.relu(conv_output)
        pooled_output = self.max_pool(conv_output).squeeze(-1)  # Get fixed-size sentence vector

        return pooled_output  # (batch_size, num_filters)

# Example usage
tokenizer = BertTokenizer.from_pretrained("bert_word_meaning_model/bert_word_meaning_model")

def encode_sentence(sentence, max_length=128):
    tokens = tokenizer(sentence, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    return tokens["input_ids"], tokens["attention_mask"]

# Sample sentence with entity indicators
sentence = ""
input_ids, attention_mask = encode_sentence(sentence)

# Model forward pass
model = SentenceEncoder()
sentence_vector = model(input_ids, attention_mask)
print("Sentence representation shape:", sentence_vector.shape)  # Should be (1, num_filters)


Some weights of the model checkpoint at bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sentence representation shape: torch.Size([1, 128])


In [14]:
sentence_vector

tensor([[0.3008, 0.4467, 0.5760, 0.2882, 0.1755, 0.4205, 0.4146, 0.2477, 0.0996,
         0.2767, 0.7258, 0.2550, 0.0848, 0.3290, 0.3271, 0.3495, 0.2258, 0.4506,
         0.0465, 0.0039, 0.4232, 0.0000, 0.0000, 0.2262, 0.1162, 0.3058, 0.3098,
         0.0000, 0.0556, 0.1764, 0.0713, 0.6215, 0.1207, 0.0696, 0.0000, 0.3101,
         0.2855, 0.0573, 0.6818, 0.0000, 0.0819, 0.3203, 0.3105, 0.2554, 0.3956,
         0.2143, 0.1763, 0.3938, 0.7179, 0.1326, 0.0000, 0.4343, 0.2079, 0.4957,
         0.3240, 0.3269, 0.3102, 0.1661, 0.4908, 0.1655, 0.0000, 0.5045, 0.3280,
         0.1764, 0.3451, 0.1552, 0.4543, 0.3516, 0.2853, 0.7005, 0.0509, 0.2396,
         0.4315, 0.3327, 0.4102, 0.2972, 0.3656, 0.1486, 0.3689, 0.5641, 0.2840,
         0.3480, 0.0237, 0.0813, 0.2338, 0.3266, 0.6664, 0.2151, 0.2413, 0.0225,
         0.0322, 0.3369, 0.0000, 0.0424, 0.2375, 0.3386, 0.2913, 0.0270, 0.3949,
         0.2037, 0.0869, 0.4615, 0.3096, 0.1846, 0.2323, 0.1199, 0.4517, 0.4533,
         0.1598, 0.2840, 0.4

In [15]:
df_malware = pd.read_csv('after_data.csv')

In [16]:
df_malware

Unnamed: 0   class                                             sha256  \
0              0    worm  009a83236c600fd7ac034973f064284cec62f86631fe96...   
1              1    worm  191828200aa2bd78f3548fb6a47407b05905efee6e83e0...   
2              2    worm  1a1937fe8a401ca27be4514acc4f811442d8201abbad2a...   
3              3    worm  21bbf357ac6d8347c68ec65fb8283a5f1185f699186959...   
4              4    worm  39563933eebcb3b09b189000e1bda255b39739792fc136...   
...          ...     ...                                                ...   
1589        1589  trojan  1fa088e4bfce3a269cd69fad5dc632fa977e6741a73f04...   
1590        1590    worm  1fad69fad4aac201134b9600e1596d327097c38046e2ff...   
1591        1591   virus  1fb2ce95769c2372560f34b00398f3681949f494bfdf1c...   
1592        1592  trojan  1fb8e85deb472dadf965916020ac9ccec0104d882eb332...   
1593        1593  trojan  1fc1cb4b3da5c338727a8cfc3f8fdb0dd955b58843b590...   

                          api_1             api_2                      api_3  \
0           GetSystemDirectoryA    IsDBCSLeadByte                 LocalAlloc   
1           GetSystemDirectoryA    IsDBCSLeadByte                 LocalAlloc   
2           GetSystemDirectoryA    IsDBCSLeadByte                 LocalAlloc   
3           GetSystemDirectoryA    IsDBCSLeadByte                 LocalAlloc   
4           GetSystemDirectoryA    IsDBCSLeadByte                 LocalAlloc   
...                         ...               ...                        ...   
1589            GlobalFindAtomW  GetModuleHandleW  DisableThreadLibraryCalls   
1590        GetSystemDirectoryA     GetVersionExW             IsDBCSLeadByte   
1591  DisableThreadLibraryCalls     GetVersionExW              GetVersionExA   
1592             VirtualAllocEx   GetStartupInfoA               GetStdHandle   
1593             LoadLibraryExW    LoadLibraryExA                 GetVersion   

                 api_4                      api_5             api_6  \
0     CreateSemaphoreW           CreateSemaphoreA    GlobalAddAtomW   
1     CreateSemaphoreW           CreateSemaphoreA    GlobalAddAtomW   
2     CreateSemaphoreW           CreateSemaphoreA    GlobalAddAtomW   
3     CreateSemaphoreW           CreateSemaphoreA    GlobalAddAtomW   
4     CreateSemaphoreW           CreateSemaphoreA    GlobalAddAtomW   
...                ...                        ...               ...   
1589     GetVersionExW              GetVersionExA  GetModuleHandleA   
1590    CompareStringW  DisableThreadLibraryCalls      CreateEventW   
1591  GetModuleHandleW           GetModuleHandleA    GetProcAddress   
1592       GetFileType               LockResource   GetCommandLineA   
1593      SetErrorMode                 LocalAlloc         LocalFree   

                       api_7  ...          api_96                    api_97  \
0                  lstrcpynW  ...  LoadLibraryExW           DuplicateHandle   
1                  lstrcpynW  ...  LoadLibraryExW           DuplicateHandle   
2                  lstrcpynW  ...  LoadLibraryExW           DuplicateHandle   
3                  lstrcpynW  ...             NaN                       NaN   
4                  lstrcpynW  ...  LoadLibraryExW           DuplicateHandle   
...                      ...  ...             ...                       ...   
1589          GetProcAddress  ...  GetProcAddress  GetWindowThreadProcessId   
1590                lstrcpyW  ...             NaN                       NaN   
1591              LocalAlloc  ...  VirtualQueryEx        GetModuleFileNameW   
1592  GetEnvironmentStringsW  ...             NaN                       NaN   
1593             GlobalAlloc  ...             NaN                       NaN   

                   api_98            api_99           api_100         api_101  \
0         DeviceIoControl    SwitchToThread        WSACleanup     FreeLibrary   
1         DeviceIoControl    SwitchToThread        WSACleanup     FreeLibrary   
2         DeviceIoControl    SwitchToThread        

In [18]:
api_seq = df_malware.iloc[0, 3:].tolist()
print(api_seq)

['GetSystemDirectoryA', 'IsDBCSLeadByte', 'LocalAlloc', 'CreateSemaphoreW', 'CreateSemaphoreA', 'GlobalAddAtomW', 'lstrcpynW', 'LoadLibraryExW', 'SearchPathW', 'CreateFileW', 'CreateFileMappingW', 'MapViewOfFileEx', 'GetSystemMetrics', 'RegisterClipboardFormatW', 'SystemParametersInfoW', 'GetDC', 'GetDeviceCaps', 'ReleaseDC', 'LocalAlloc', 'GetSysColor', 'GetSysColorBrush', 'GetStockObject', 'GetSystemMetrics', 'LoadCursorW', 'RegisterClassW', 'RegisterClassExW', 'LoadLibraryExW', 'LoadLibraryW', 'GetCommandLineA', 'GetStartupInfoA', 'LockResource', 'GetModuleFileNameA', 'IsBadWritePtr', 'RegisterClipboardFormatW', 'SystemParametersInfoW', 'GetSystemMetrics', 'LocalAlloc', 'GetSysColor', 'GetSysColorBrush', 'GetStockObject', 'LoadLibraryW', 'LoadLibraryExW', 'LoadCursorW', 'RegisterClassW', 'GetKeyboardType', 'GetCommandLineA', 'GetStartupInfoA', 'GetVersion', 'GetModuleFileNameA', 'lstrcpynA', 'GetThreadLocale', 'GetLocaleInfoW', 'GetLocaleInfoA', 'lstrlenA', 'LoadLibraryExW', 'Search

In [19]:
api_encodings = {}

In [20]:
for i in api_seq:
    input_ids, attention_mask = encode_sentence(i)
    model = SentenceEncoder()
    sentence_vector = model(input_ids, attention_mask)
    api_encodings[i] = sentence_vector
    print(f"{i} : {sentence_vector}")

Some weights of the model checkpoint at bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


GetSystemDirectoryA : tensor([[0.3700, 0.5608, 0.3364, 0.3082, 0.4579, 0.4216, 0.4638, 0.3152, 0.3051,
         0.3352, 0.4057, 0.3673, 0.4006, 0.2240, 0.1831, 0.0235, 0.2988, 0.3304,
         0.2721, 0.2330, 0.2190, 0.1549, 0.4104, 0.3493, 0.4457, 0.1178, 0.4677,
         0.1797, 0.0000, 0.4017, 0.4426, 0.2948, 0.1348, 0.2106, 0.6146, 0.4306,
         0.0000, 0.0000, 0.1222, 0.1061, 0.2629, 0.0800, 0.2913, 0.3903, 0.2431,
         0.1963, 0.3516, 0.4667, 0.4200, 0.2162, 0.1353, 0.2261, 0.2147, 0.4162,
         0.0754, 0.1437, 0.6058, 0.2081, 0.3427, 0.3530, 0.1277, 0.1962, 0.4860,
         0.3180, 0.0341, 0.2615, 0.4837, 0.0766, 0.1465, 0.0856, 0.5002, 0.3475,
         0.1937, 0.4223, 0.0753, 0.3485, 0.3557, 0.3787, 0.1824, 0.3897, 0.3558,
         0.0000, 0.3337, 0.3009, 0.2456, 0.0000, 0.2840, 0.0894, 0.0000, 0.3026,
         0.0523, 0.2015, 0.3765, 0.4826, 0.3551, 0.3991, 0.8802, 0.0405, 0.1825,
         0.3429, 0.0511, 0.5003, 0.1266, 0.5862, 0.4228, 0.2579, 0.1056, 0.3798,
      

Some weights of the model checkpoint at bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


IsDBCSLeadByte : tensor([[0.2316, 0.2774, 0.1943, 0.1398, 0.3043, 0.3922, 0.3911, 0.1857, 0.5804,
         0.2832, 0.6518, 0.2279, 0.4148, 0.3166, 0.1661, 0.4550, 0.1850, 0.2371,
         0.0427, 0.1233, 0.4659, 0.0630, 0.3760, 0.3394, 0.5051, 0.3296, 0.3774,
         0.2985, 0.7153, 0.0726, 0.0552, 0.0000, 0.0000, 0.3754, 0.2059, 0.4303,
         0.0114, 0.0000, 0.1927, 0.3478, 0.0037, 0.4171, 0.2240, 0.0000, 0.2387,
         0.2699, 0.1578, 0.4137, 0.0635, 0.0000, 0.2811, 0.1005, 0.3377, 0.3633,
         0.5462, 0.0024, 0.1559, 0.0750, 0.1922, 0.3615, 0.0206, 0.0000, 0.3866,
         0.2496, 0.3303, 0.2370, 0.1269, 0.3986, 0.2629, 0.1393, 0.0000, 0.6321,
         0.4285, 0.6068, 0.1423, 0.5756, 0.4806, 0.3168, 0.3977, 0.2187, 0.2408,
         0.1344, 0.1563, 0.0371, 0.4787, 0.0000, 0.1823, 0.3606, 0.0000, 0.4890,
         0.1714, 0.2756, 0.2422, 0.5729, 0.4478, 0.1203, 0.3302, 0.4669, 0.2101,
         0.3481, 0.3582, 0.3647, 0.0000, 0.7616, 0.3402, 0.0000, 0.1455, 0.4358,
         0.

Some weights of the model checkpoint at bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

ResetEvent : tensor([[0.0000, 0.3804, 0.4219, 0.2734, 0.4135, 0.3623, 0.3112, 0.0508, 0.3291,
         0.2385, 0.3443, 0.4102, 0.3160, 0.2714, 0.1167, 0.4977, 0.4729, 0.0000,
         0.1751, 0.0754, 0.6285, 0.3126, 0.2399, 0.5850, 0.0000, 0.2304, 0.4771,
         0.1959, 0.1579, 0.0000, 0.3267, 0.6949, 0.5072, 0.3848, 0.4057, 0.0000,
         0.3036, 0.4855, 0.1075, 0.1576, 0.1799, 0.1357, 0.2789, 0.4809, 0.1093,
         0.2781, 0.1065, 0.3973, 0.4177, 0.3251, 0.1388, 0.2653, 0.3798, 0.2492,
         0.3350, 0.1646, 0.2587, 0.5699, 0.6070, 0.0593, 0.0847, 0.0543, 0.3686,
         0.3300, 0.2622, 0.1717, 0.1318, 0.0000, 0.1071, 0.3807, 0.0805, 0.1257,
         0.0217, 0.4216, 0.1682, 0.0901, 0.2498, 0.4697, 0.6178, 0.0000, 0.2779,
         0.3458, 0.4967, 0.1755, 0.1043, 0.3830, 0.0000, 0.0034, 0.0504, 0.2302,
         0.2930, 0.4579, 0.2446, 0.0499, 0.5749, 0.1684, 0.1385, 0.4150, 0.2379,
         0.0119, 0.2360, 0.0870, 0.5343, 0.3445, 0.6896, 0.0000, 0.0000, 0.0395,
         0.1342

Some weights of the model checkpoint at bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


VirtualFreeEx : tensor([[0.2209, 0.3013, 0.1793, 0.2760, 0.0000, 0.0910, 0.3946, 0.0000, 0.1089,
         0.2847, 0.1573, 0.3010, 0.4112, 0.3092, 0.4269, 0.0506, 0.5087, 0.2059,
         0.2270, 0.2112, 0.2144, 0.2060, 0.3047, 0.0000, 0.2042, 0.1893, 0.0735,
         0.2157, 0.2073, 0.5112, 0.4463, 0.4099, 0.2177, 0.4877, 0.1171, 0.2314,
         0.2402, 0.4780, 0.3565, 0.1932, 0.0000, 0.1710, 0.2896, 0.2053, 0.0000,
         0.2214, 0.3160, 0.2845, 0.3182, 0.2833, 0.5217, 0.3645, 0.4365, 0.3832,
         0.0445, 0.2777, 0.5227, 0.2479, 0.1563, 0.6497, 0.1104, 0.0950, 0.3321,
         0.2892, 0.1648, 0.4128, 0.1643, 0.2853, 0.6365, 0.1052, 0.4294, 0.1867,
         0.4451, 0.2266, 0.3786, 0.2855, 0.1276, 0.1374, 0.3200, 0.2384, 0.1688,
         0.3427, 0.5148, 0.3051, 0.4407, 0.0000, 0.3404, 0.5505, 0.2222, 0.2891,
         0.4025, 0.3717, 0.3843, 0.2795, 0.0573, 0.1030, 0.5515, 0.2674, 0.3624,
         0.5194, 0.1295, 0.1106, 0.2140, 0.0765, 0.1458, 0.2563, 0.3330, 0.4869,
         0.4

Some weights of the model checkpoint at bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


UnregisterClassW : tensor([[0.0000, 0.4562, 0.2533, 0.1187, 0.2017, 0.2791, 0.1837, 0.1747, 0.2680,
         0.2412, 0.3880, 0.3598, 0.4244, 0.4330, 0.2451, 0.0000, 0.4334, 0.0000,
         0.0958, 0.1420, 0.3484, 0.2018, 0.0491, 0.4311, 0.0612, 0.0313, 0.4275,
         0.3739, 0.3017, 0.0000, 0.2517, 0.6916, 0.5195, 0.3322, 0.1561, 0.8637,
         0.2854, 0.2220, 0.6900, 0.3928, 0.1918, 0.2662, 0.5363, 0.1215, 0.1260,
         0.4183, 0.4531, 0.0000, 0.5481, 0.0560, 0.2154, 0.0149, 0.7403, 0.1821,
         0.3973, 0.4232, 0.3465, 0.1160, 0.4770, 0.1870, 0.2922, 0.3902, 0.3812,
         0.4903, 0.0010, 0.5789, 0.0000, 0.5096, 0.4564, 0.0000, 0.0000, 0.3406,
         0.2364, 0.4797, 0.3712, 0.3571, 0.0081, 0.3121, 0.2022, 0.3901, 0.1665,
         0.4334, 0.3251, 0.3069, 0.3971, 0.6036, 0.1537, 0.3483, 0.0000, 0.1078,
         0.0181, 0.2578, 0.1138, 0.2823, 0.3189, 0.2909, 0.1026, 0.1799, 0.5688,
         0.1709, 0.2025, 0.1176, 0.0659, 0.2773, 0.1838, 0.2388, 0.0000, 0.3234,
         

In [ ]:
api_encodings

{'GetSystemDirectoryA': tensor([[0.1791, 0.0897, 0.4013, 0.0913, 0.3151, 0.1763, 0.2815, 0.0365, 0.4221,
          0.4213, 0.3429, 0.1986, 0.0061, 0.3363, 0.3899, 0.2745, 0.4443, 0.0000,
          0.2221, 0.7003, 0.3193, 0.1772, 0.1659, 0.4850, 0.2074, 0.4410, 0.0454,
          0.7320, 0.1208, 0.1634, 0.2454, 0.1600, 0.5079, 0.1913, 0.4262, 0.6356,
          0.0000, 0.6557, 0.5203, 0.1328, 0.2936, 0.0309, 0.3477, 0.2330, 0.1146,
          0.4291, 0.3936, 0.0000, 0.1235, 0.0241, 0.2558, 0.0000, 0.6882, 0.4521,
          0.0866, 0.0553, 0.0099, 0.3571, 0.3162, 0.0000, 0.2845, 0.2768, 0.3188,
          0.3708, 0.0000, 0.3231, 0.5861, 0.7029, 0.1408, 0.3252, 0.0000, 0.3491,
          0.0000, 0.1961, 0.4351, 0.2685, 0.2551, 0.1990, 0.2989, 0.1004, 0.1775,
          0.3583, 0.3025, 0.3262, 0.0527, 0.2519, 0.3159, 0.1675, 0.5502, 0.6872,
          0.0000, 0.3727, 0.4296, 0.3348, 0.4670, 0.0000, 0.0130, 0.3149, 0.5536,
          0.2844, 0.4304, 0.2721, 0.2217, 0.0530, 0.0927, 0.3196, 0.6662, 0

In [ ]:
api_encodings2 = {}
for i in api_seq:
    input_ids, attention_mask = encode_sentence(i,max_length=21)
    model = SentenceEncoder()
    sentence_vector = model(input_ids, attention_mask)
    api_encodings2[i] = sentence_vector
    print(f"{i} : {sentence_vector}")

GetSystemDirectoryA : tensor([[0.1702, 0.0948, 0.3632, 0.2980, 0.0000, 0.1676, 0.0815, 0.1435, 0.3647,
         0.1241, 0.2442, 0.2072, 0.0000, 0.2428, 0.0000, 0.3141, 0.0204, 0.4215,
         0.3480, 0.3281, 0.1594, 0.2672, 0.2343, 0.2005, 0.0000, 0.3530, 0.2317,
         0.3703, 0.4326, 0.1178, 0.0000, 0.0000, 0.0000, 0.0755, 0.1190, 0.0511,
         0.1724, 0.1242, 0.3329, 0.4753, 0.2282, 0.3451, 0.2723, 0.1274, 0.4604,
         0.3927, 0.2075, 0.0703, 0.2507, 0.1654, 0.2375, 0.3210, 0.1090, 0.0813,
         0.1255, 0.0000, 0.2225, 0.5715, 0.0000, 0.0296, 0.4663, 0.1082, 0.2556,
         0.3168, 0.2251, 0.2397, 0.0000, 0.3537, 0.3356, 0.5153, 0.1166, 0.3317,
         0.1883, 0.3022, 0.4325, 0.2898, 0.1416, 0.1108, 0.6034, 0.0949, 0.2483,
         0.0749, 0.2407, 0.0296, 0.3713, 0.3276, 0.0170, 0.5291, 0.3401, 0.1795,
         0.0620, 0.2000, 0.0897, 0.3333, 0.3453, 0.0495, 0.3450, 0.3761, 0.3235,
         0.3772, 0.3910, 0.3988, 0.4989, 0.1951, 0.5296, 0.0798, 0.0000, 0.1925,
      

In [ ]:
api_encodings2

In [ ]:
sentence_vector

tensor([[0.4907, 0.4222, 0.0000, 0.3999, 0.4758, 0.3901, 0.0000, 0.5541, 0.0505,
         0.6262, 0.1223, 0.3879, 0.4151, 0.2004, 0.2518, 0.2718, 0.2575, 0.0865,
         0.4626, 0.1429, 0.0889, 0.4190, 0.4350, 0.4156, 0.2874, 0.0000, 0.5432,
         0.2532, 0.0166, 0.1277, 0.2530, 0.4031, 0.0128, 0.4125, 0.3263, 0.3251,
         0.2090, 0.4125, 0.2497, 0.5074, 0.2928, 0.3767, 0.7107, 0.0459, 0.4249,
         0.2378, 0.3416, 0.2791, 0.2498, 0.3475, 0.5838, 0.5298, 0.3248, 0.4576,
         0.2675, 0.1830, 0.0000, 0.5551, 0.0805, 0.3740, 0.3836, 0.2753, 0.5755,
         0.7983, 0.1613, 0.3692, 0.0826, 0.3429, 0.2091, 0.4378, 0.3228, 0.3954,
         0.1943, 0.4002, 0.2969, 0.2972, 0.2551, 0.2124, 0.3899, 0.0845, 0.7423,
         0.1856, 0.5603, 0.1537, 0.1061, 0.1910, 0.3674, 0.3680, 0.1135, 0.1624,
         0.3041, 0.1200, 0.3008, 0.0000, 0.2311, 0.0000, 0.6468, 0.1770, 0.0000,
         0.4106, 0.4127, 0.1586, 0.2548, 0.3382, 0.2424, 0.1789, 0.4270, 0.4180,
         0.3731, 0.2871, 0.0

In [21]:
tokens = []
for i in api_seq:
    tokens.append(tokenizer.tokenize(i))
print(tokens)

[['gets', '##yst', '##em', '##di', '##re', '##ctor', '##ya'], ['is', '##db', '##cs', '##lea', '##db', '##yte'], ['local', '##all', '##oc'], ['creates', '##ema', '##ph', '##ore', '##w'], ['creates', '##ema', '##ph', '##ore', '##a'], ['global', '##ad', '##da', '##tom', '##w'], ['l', '##st', '##rc', '##py', '##n', '##w'], ['load', '##lib', '##rar', '##ye', '##x', '##w'], ['search', '##path', '##w'], ['create', '##fi', '##le', '##w'], ['create', '##fi', '##lem', '##app', '##ing', '##w'], ['map', '##view', '##off', '##ile', '##ex'], ['gets', '##yst', '##em', '##metric', '##s'], ['register', '##cl', '##ip', '##board', '##form', '##at', '##w'], ['system', '##para', '##meter', '##sin', '##fo', '##w'], ['get', '##dc'], ['get', '##dev', '##ice', '##cap', '##s'], ['released', '##c'], ['local', '##all', '##oc'], ['gets', '##ys', '##color'], ['gets', '##ys', '##color', '##brush'], ['gets', '##tock', '##ob', '##ject'], ['gets', '##yst', '##em', '##metric', '##s'], ['load', '##cu', '##rso', '##r', '#

DATA PROCESSING

In [22]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

class ApiSequenceDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=64):
        """
        Dataset for API call sequences

        Args:
            data_path: Path to the CSV file containing API sequences
            tokenizer: BERT tokenizer
            max_length: Maximum sequence length (for padding/truncation)
        """
        self.data = pd.read_csv(data_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Extract API sequence columns (all columns starting with 'api_seq_')
        self.api_cols = [col for col in self.data.columns if col.startswith('api_seq_')]

        # Create class labels mapping
        self.classes = self.data['class_name'].unique()
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Extract API sequence for this sample (handling variable length)
        api_sequence = []
        for col in self.api_cols:
            if pd.notna(row[col]):  # Only include non-NaN values
                api_sequence.append(str(row[col]))

        # Convert class name to index
        label = self.class_to_idx[row['class_name']]

        # Tokenize the API sequence
        # We join API calls with spaces to treat them as words
        api_sequence_str = " ".join(api_sequence)

        # Tokenize using BERT tokenizer
        encoding = self.tokenizer(
            api_sequence_str,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Remove batch dimension added by tokenizer
        for key in encoding:
            encoding[key] = encoding[key].squeeze(0)

        # Add sample metadata
        encoding['sha256'] = row['sha256']
        encoding['label'] = torch.tensor(label)
        encoding['sequence_length'] = min(len(api_sequence), self.max_length)

        return encoding


Collacation Function

In [23]:
def collate_api_sequences(batch):
    """
    Collate function to handle variable length API sequences in a batch
    """
    # Get the maximum sequence length in this batch
    max_len = max([sample['sequence_length'] for sample in batch])

    # Initialize batch dictionaries
    batch_encoding = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': [],
        'sha256': [],
        'label': [],
        'sequence_length': []
    }

    # Collect samples
    for sample in batch:
        # Truncate or pad to the max length in this batch
        # We already padded to max_length in the dataset, but we might need to adjust
        batch_encoding['input_ids'].append(sample['input_ids'][:max_len])
        batch_encoding['attention_mask'].append(sample['attention_mask'][:max_len])
        if 'token_type_ids' in sample:
            batch_encoding['token_type_ids'].append(sample['token_type_ids'][:max_len])

        # Add metadata
        batch_encoding['sha256'].append(sample['sha256'])
        batch_encoding['label'].append(sample['label'])
        batch_encoding['sequence_length'].append(sample['sequence_length'])

    # Convert lists to tensors
    batch_encoding['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        batch_encoding['input_ids'], batch_first=True, padding_value=0
    )
    batch_encoding['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        batch_encoding['attention_mask'], batch_first=True, padding_value=0
    )
    if batch_encoding['token_type_ids']:
        batch_encoding['token_type_ids'] = torch.nn.utils.rnn.pad_sequence(
            batch_encoding['token_type_ids'], batch_first=True, padding_value=0
        )

    batch_encoding['label'] = torch.stack(batch_encoding['label'])
    batch_encoding['sequence_length'] = torch.tensor(batch_encoding['sequence_length'])

    return batch_encoding


Modified Architecture

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

class Proto_CATT_BERT_API(nn.Module):
    """
    Context Attention-based Prototypical Networks with BERT for API Sequence Classification
    Modified to handle variable-length API call sequences
    """
    def __init__(self, bert_model='bert-base-uncased', hidden_size=768, conv_window_size=3, num_heads=12, num_classes=2):
        super(Proto_CATT_BERT_API, self).__init__()

        # BERT embedding layer
        self.bert = BertModel.from_pretrained(bert_model)
        self.hidden_size = hidden_size

        # CNN encoding layer
        self.conv = nn.Conv1d(hidden_size, hidden_size, kernel_size=conv_window_size,
                             padding=(conv_window_size-1)//2)

        # Number of attention heads
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        # Classification layer (if not using prototypical networks)
        self.classifier = nn.Linear(hidden_size, num_classes)

        assert self.head_dim * num_heads == hidden_size, "hidden_size must be divisible by num_heads"

    def sentence_encoder(self, input_ids, attention_mask=None, token_type_ids=None):
        """
        Encode a sentence using BERT and CNN
        """
        # Get BERT embeddings
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        word_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)

        # Apply CNN for feature extraction
        # Transpose for conv1d which expects (batch_size, channels, seq_len)
        word_embeddings = word_embeddings.transpose(1, 2)  # (batch_size, hidden_size, seq_len)
        conv_output = self.conv(word_embeddings)  # (batch_size, hidden_size, seq_len)

        # Apply max pooling over sequence length
        # This handles variable length by taking the maximum feature value across the sequence
        sentence_embedding = F.max_pool1d(conv_output, kernel_size=conv_output.size(2))
        sentence_embedding = sentence_embedding.squeeze(2)  # (batch_size, hidden_size)

        return sentence_embedding

    def context_attention(self, support_set):
        """
        Apply context attention to the support set

        Implementation of equation: softmax(S*S^T/sqrt(d_w))*S with multi-head attention
        """
        batch_size, num_instances, _ = support_set.size()

        # Split into multiple heads
        support_set_heads = support_set.view(batch_size, num_instances, self.num_heads, self.head_dim)
        support_set_heads = support_set_heads.permute(0, 2, 1, 3)  # (batch_size, num_heads, num_instances, head_dim)

        # Calculate attention for each head
        attention_scores = torch.matmul(support_set_heads, support_set_heads.transpose(-2, -1))
        attention_scores = attention_scores / (self.head_dim ** 0.5)  # Scale by sqrt(d_w)
        attention_weights = F.softmax(attention_scores, dim=-1)

        # Apply attention weights
        weighted_heads = torch.matmul(attention_weights, support_set_heads)

        # Combine heads
        weighted_heads = weighted_heads.permute(0, 2, 1, 3)  # (batch_size, num_instances, num_heads, head_dim)
        weighted_support = weighted_heads.reshape(batch_size, num_instances, self.hidden_size)

        return weighted_support

    def forward_prototypical(self, query_inputs, support_inputs, n_way, k_shot):
        """
        Forward pass for few-shot learning using prototypical networks
        """
        # Process query instances
        query_embedding = self.sentence_encoder(
            query_inputs['input_ids'],
            attention_mask=query_inputs.get('attention_mask', None),
            token_type_ids=query_inputs.get('token_type_ids', None)
        )  # (batch_size, hidden_size)

        # Process support set instances
        batch_size = support_inputs['input_ids'].size(0)
        support_instance_count = support_inputs['input_ids'].size(1)  # n_way * k_shot

        # Reshape for processing each instance individually
        flat_input_ids = support_inputs['input_ids'].view(-1, support_inputs['input_ids'].size(-1))
        flat_attention_mask = None
        if 'attention_mask' in support_inputs:
            flat_attention_mask = support_inputs['attention_mask'].view(-1, support_inputs['attention_mask'].size(-1))
        flat_token_type_ids = None
        if 'token_type_ids' in support_inputs:
            flat_token_type_ids = support_inputs['token_type_ids'].view(-1, support_inputs['token_type_ids'].size(-1))

        # Encode all support instances
        flat_support_embeddings = self.sentence_encoder(
            flat_input_ids,
            attention_mask=flat_attention_mask,
            token_type_ids=flat_token_type_ids
        )  # (batch_size * support_instance_count, hidden_size)

        # Reshape back
        support_embeddings = flat_support_embeddings.view(batch_size, support_instance_count, -1)

        # Apply context attention to highlight important instances
        weighted_support = self.context_attention(support_embeddings)

        # Reshape support set for n-way k-shot
        support_embeddings = weighted_support.view(batch_size, n_way, k_shot, -1)

        # Calculate prototypes by averaging the k instances for each class
        prototypes = support_embeddings.mean(dim=2)  # (batch_size, n_way, hidden_size)

        # Calculate Euclidean distances between query and prototypes
        query_expanded = query_embedding.unsqueeze(1)  # (batch_size, 1, hidden_size)
        distances = torch.sum((query_expanded - prototypes) ** 2, dim=2)  # (batch_size, n_way)

        # Convert distances to logits (negative distances)
        logits = -distances

        return logits

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        """
        Standard forward pass for classification (non-few-shot)
        """
        # Get sentence embeddings
        embeddings = self.sentence_encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # Get logits through the classifier
        logits = self.classifier(embeddings)

        # Calculate loss if labels are provided
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            return loss, logits

        return logits


Training Code

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load the raw CSV
df = pd.read_csv('after_data.csv')

# 2. Drop pandas auto‑index if present
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# 3. Combine all api_* columns into a single comma‑separated string
api_cols = [c for c in df.columns if c.startswith('api_')]
df['api_sequence'] = df[api_cols] \
    .apply(lambda row: ','.join(x for x in row.astype(str) if x and x.lower() != 'nan'), axis=1)

# 4. Keep the sha256, api_sequence, and class (renaming 'class' to 'label')
df = df[['sha256', 'api_sequence', 'class']]

# 5. Rename the 'class' column to 'label'
df = df.rename(columns={'class': 'label'})

# 6. (Optional) Drop any labels with only one example to allow stratification
counts = df['label'].value_counts()
singletons = counts[counts < 2].index.tolist()
if singletons:
    print(f"Dropping singleton labels: {singletons}")
    df = df[~df['label'].isin(singletons)]

# 7. Split 80% train / 20% validation, stratifying on the label
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)

# 8. Save to CSV with sha256 and label included
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

# Confirm the columns
print(f"train.csv rows: {len(train_df)}, val.csv rows: {len(val_df)}")
print("Columns in train.csv:", list(train_df.columns))  # Ensure 'label' is in the final columns


Dropping singleton labels: ['UDS:DangerousObject', 'hacktool', 'rootkit']
train.csv rows: 1272, val.csv rows: 319
Columns in train.csv: ['sha256', 'api_sequence', 'label']


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
import numpy as np
import random
import pandas as pd
from tqdm import tqdm

# =========================
# 1. Random Seed
# =========================
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================
# 2. Dataset & Collate
# =========================
class ApiSequenceDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length):
        self.data = pd.read_csv(data_path)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.classes = sorted(self.data['label'].unique())
        self.label2idx = {label: idx for idx, label in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence = self.data.iloc[idx]['api_sequence']
        label = self.data.iloc[idx]['label']
        encoding = self.tokenizer(
            str(sequence),
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.label2idx[label], dtype=torch.long)
        }

def collate_api_sequences(batch):
    return {
        'input_ids': torch.stack([item['input_ids'] for item in batch]),
        'attention_mask': torch.stack([item['attention_mask'] for item in batch]),
        'label': torch.stack([item['label'] for item in batch])
    }

# =========================
# 3. Tokenizer
# =========================
tokenizer = BertTokenizer.from_pretrained('/content/bert_word_meaning_model/bert_word_meaning_model')
special_tokens = ['<PAD>', '<UNK>']
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# =========================
# 4. Datasets & Loaders
# =========================
train_dataset = ApiSequenceDataset(
    data_path='train.csv',
    tokenizer=tokenizer,
    max_length=64
)
val_dataset = ApiSequenceDataset(
    data_path='val.csv',
    tokenizer=tokenizer,
    max_length=64
)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_api_sequences,
    drop_last=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=collate_api_sequences
)

# =========================
# 5. Model Setup
# =========================
num_classes = len(train_dataset.classes)
model = Proto_CATT_BERT_API(
    bert_model='/content/bert_word_meaning_model/bert_word_meaning_model',
    hidden_size=768,
    conv_window_size=3,
    num_heads=12,
    num_classes=num_classes
)
model.bert.resize_token_embeddings(len(tokenizer))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# =========================
# 6. Optimizer & Scheduler
# =========================
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
total_steps = len(train_loader) * 10  # 10 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# =========================
# 7. Training Loop
# =========================
def train_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=10):
    best_accuracy = 0
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        # Training
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()
            loss, _ = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                token_type_ids=None,
                labels=batch['label']
            )
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # Validation
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    token_type_ids=None
                )
                # If model returns tuple (logits, prototypes)
                logits = outputs[0] if isinstance(outputs, tuple) else outputs
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                labels = batch['label'].cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels)
        accuracy = accuracy_score(all_labels, all_preds)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print("F1-score: {:.4f}".format(f1_score(all_labels, all_preds, average='weighted')))
        print("Precision: {:.4f}".format(precision_score(all_labels, all_preds, average='weighted')))
        print("Recall: {:.4f}".format(recall_score(all_labels, all_preds, average='weighted')))
        print("\nClassification Report:")
        print(classification_report(
            all_labels,
            all_preds,
            target_names=[train_dataset.classes[i] for i in range(num_classes)]
        ))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_api_model_trained.pt')
            print(f"Model saved with accuracy: {best_accuracy:.4f}")
    return model

# =========================
# 8. Train!
# =========================
trained_model = train_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=10)


Some weights of the model checkpoint at /content/bert_word_meaning_model/bert_word_meaning_model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/10


Training: 100%|██████████| 79/79 [00:21<00:00,  3.72it/s]


Average training loss: 1.2929


Validation: 100%|██████████| 20/20 [00:02<00:00,  8.05it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

...

Training: 100%|██████████| 79/79 [00:21<00:00,  3.70it/s]


Average training loss: 0.6287


Validation: 100%|██████████| 20/20 [00:02<00:00,  9.08it/s]

Validation Accuracy: 0.7931
F1-score: 0.7611
Precision: 0.7629
Recall: 0.7931

Classification Report:
              precision    recall  f1-score   support

    backdoor       0.00      0.00      0.00         7
        hoax       0.48      0.91      0.62        11
 notdetected       0.38      0.12      0.18        26
      packed       1.00      0.80      0.89        30
      trojan       0.82      0.92      0.87       178
       virus       0.79      0.91      0.84        54
        worm       0.75      0.23      0.35        13

    accuracy                           0.79       319
   macro avg       0.60      0.55      0.54       319
weighted avg       0.76      0.79      0.76       319




/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/